In [ ]:
from bs4 import BeautifulSoup
import base64
import wget
import requests
import unicodecsv as csv
import datetime

In [ ]:
total_articles = [] # it will save all publications

In [ ]:
## get days
iterator_day = datetime.date(2019, 9, 3)
last_publication = datetime.date(2014, 1, 1)

In [ ]:
array_days = [str(iterator_day)]
while(iterator_day >= last_publication):
    iterator_day = iterator_day - datetime.timedelta(days=1)
    array_days.append(str(iterator_day))

In [ ]:
def save_csv(articles):
    with open('el_comercio.csv', 'wb') as f:
            dict_writer = csv.DictWriter(
                f,
                [
                    'Titulo',
                    'Resumen',
                    'Categoria', 
                    'Autor', 
                    'Fecha', 
                    'Contenido', 
                    'Etiquetas',
                    'Enlace'
                ],
                encoding='utf-8')
            dict_writer.writeheader()
            dict_writer.writerows(articles)

In [ ]:
def get_title(content_soup):
    if content_soup.find('h1', {'class': 'news-title'}) is not None:
        return content_soup.find('h1', {'class': 'news-title'}).text
    
    return;

In [ ]:
def get_summary(content_soup):
    if content_soup.find('h2', {'class': 'news-summary'}) is not None:
        return content_soup.find('h2', {'class': 'news-summary'}).text
    
    return;

In [ ]:
def get_category(content_soup):
    if content_soup.find('h3', {'class': 'news-category fija'}) is not None:
        if content_soup.find('h3', {'class': 'news-category fija'}).find('a') is not None:
            return content_soup.find('h3', {'class': 'news-category fija'}).find('a').text
    
    return;

In [ ]:
def get_author(content_soup):
    if content_soup.find('div', {'class': 'news-author-date'}) is not None:
        if content_soup.find('div', {'class': 'news-author-date'}).find('a') is not None:
            if content_soup.find('div', {'class': 'news-author-date'}).find('a').find('span') is not None:
                return content_soup.find('div', {'class': 'news-author-date'}).find('a').find('span').text
    
    return;

In [ ]:
def get_date(content_soup):
    if content_soup.find('div', {'class': 'news-author-date'}) is not None:
        if content_soup.find('div', {'class': 'news-author-date'}).find('time') is not None:
            return content_soup.find('div', {'class': 'news-author-date'}).find('time')['datetime']
    
    return;

In [ ]:
def get_content(content_soup):
    if content_soup.find('div', {'class': 'news-text-content'}) is not None:
        if len(content_soup.find('div', {'class': 'news-text-content'}).findAll('p', {'class': 'parrafo'})) > 0:
            return content_soup.find('div', {'class': 'news-text-content'}).findAll('p', {'class': 'parrafo'})
    
    return [];

In [ ]:
def get_tags(content_soup):
    if len(content_soup.findAll('h2', {'class': 'tags-item'})) > 0:
        return content_soup.findAll('h2', {'class': 'tags-item'})
    
    return [];

In [ ]:
def interate_articles_and_save(articles):
    cont = 0
    for article in articles:
        print('-------------------- Article ----------------------')
        url_article = 'https://elcomercio.pe' + article.find('a', {'class': 'page-link'})['href']
        content_article = requests.get(url_article).content
        soup_article = BeautifulSoup(content_article,'lxml')
        print("URL Article===>" + str(url_article))
        
        title = get_title(soup_article)
        summary = get_summary(soup_article)
        category = get_category(soup_article)
        author = get_author(soup_article)
        date = get_date(soup_article)
        text_content = get_content(soup_article)
        content_article = []
        for text in text_content:    
            content_article.append(text.text)

        tags_article = get_tags(soup_article)
        tags = []
        for tag in tags_article:
            tags.append(tag.text)

        total_articles.extend([
            {
                'Titulo':str(title), 
                'Resumen': str(summary), 
                'Categoria':str(category), 
                'Autor':str(author),
                'Fecha':str(date),
                'Contenido':str(content_article),
                'Etiquetas':str(tags),
                'Enlace':url_article
            }
        ])
        save_csv(total_articles)
        cont = cont + 1
        print('-----------------------------------  ' + str(cont*100/len(articles)) + "%")
        print('-------------------- End Article ----------------------')

In [ ]:
def iterate_publications(days):
    for day in days:
        print('-------------------- Day ----------------------')
        url_day = 'https://elcomercio.pe/archivo/todas/' + day
        content_day = requests.get(url_day).content
        soup_day = BeautifulSoup(content_day,'lxml')
        print("URL Day===>" + str(url_day))
        
        articles_day = soup_day.findAll('article', {'class' : 'section-flow'})
        try:
            interate_articles_and_save(articles_day)
        except:
            print('Error para obtener pagina====> ' + url_day)
        print('-------------------- End Day ----------------------')

In [ ]:
iterate_publications(array_days)